In [10]:
# importing libraries
import numpy as np
import pandas as pd
from PIL import Image
import torch
import tensorflow as tf
import torchvision
from torchvision import transforms
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import pickle as pkl
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import sklearn
import seaborn as sns
import cv2
import skimage.exposure
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
import PIL
from tqdm.notebook import tqdm, trange
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
RezNetTransform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = RezNetTransform):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        filename = str(self.df.id[index])+'.jpg'
        label = self.df.stable[index]
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        sample = {'image': image, 'stable': label}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        return sample

In [5]:
dataset = CustomDataset("blocks-labels.csv", './data')

In [6]:
train_size = int(0.8 * len(dataset))
test_size = 51276 - train_size
train, test = torch.utils.data.random_split(dataset, [train_size, test_size])

In [7]:
train_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True)

In [8]:
resnet = models.resnet18(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)
resnet = resnet.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9)
print(criterion, optimizer)

losses = []
for epoch in range(1):  # loop over the dataset
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_loader)):
        # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        inputs, labels = data['image'].to(device), data['stable'].to(device)
        optimizer.zero_grad()
        # forward + backward + optimize
        output = resnet(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:   
            print(loss.item())
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
print('Finished Training of RezNet')

CrossEntropyLoss() SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


1.611769199371338
[1,   200] loss: 3.275
1.820715308189392
[1,   400] loss: 1.153
3.028890609741211
[1,   600] loss: 1.225
1.2305315732955933
[1,   800] loss: 0.936
0.17305225133895874
[1,  1000] loss: 1.036
0.6461089253425598
[1,  1200] loss: 0.831
0.707642138004303
[1,  1400] loss: 0.701
0.6959787607192993
[1,  1600] loss: 0.713
0.6695823669433594
[1,  1800] loss: 0.704
0.7003813982009888
[1,  2000] loss: 0.705
0.7299576997756958
[1,  2200] loss: 0.705
0.8355540037155151
[1,  2400] loss: 0.698
0.589571475982666
[1,  2600] loss: 0.773
0.7278444766998291
[1,  2800] loss: 0.735
0.7028849124908447
[1,  3000] loss: 0.694
0.7017541527748108
[1,  3200] loss: 0.701
0.6365395784378052
[1,  3400] loss: 0.695
0.7314634323120117
[1,  3600] loss: 0.698
0.6537660956382751
[1,  3800] loss: 0.693
0.7422634959220886
[1,  4000] loss: 0.698
0.6801863312721252
[1,  4200] loss: 0.692
0.7080267667770386
[1,  4400] loss: 0.699
0.6561776995658875
[1,  4600] loss: 0.698
0.6805788278579712
[1,  4800] loss: 0.